In [8]:
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import os
import albumentations as A
import torch
import torch.nn as nn
from torchvision.models.segmentation import fcn_resnet50
from albumentations.pytorch import ToTensorV2
from pathlib import Path

> ## 사용법

1. `./data`로 current working directory 변경
2. train 데이터 바꾸고 싶으면 `train_dir = True`, eval 데이터 바꾸고 싶으면 `train_dir = False`
2. 코드 실행
3. `./data/train/images_segmented/...`에 segmented 이미지들 원래랑 똑같은 구조로 저장됨

In [18]:
train_dir = True
target_dir = 'train' if train_dir else 'eval'

In [19]:
segmented_img_dir_path = './{}/images_segmented'.format(target_dir)
orig_img_dir_path = './{}/images'.format(target_dir)
if not os.path.isdir(segmented_img_dir_path):
    os.mkdir(segmented_img_dir_path)

In [20]:
model = fcn_resnet50(pretrained=True, progress=False)
model = model.eval()

In [21]:
def segment_img(img):
    transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    ),
    ToTensorV2(),
    ])
    
    original_img = img

    img = transform(image=np.array(img))['image']
    img_batch = img.unsqueeze(0)

    output = model(img_batch)['out']
    normalized_masks = torch.nn.functional.softmax(output, dim=1)

    person_mask = normalized_masks[0,15]
    masked_img = person_mask.detach().numpy()
    
    boundary = 5
    xmax = masked_img.shape[0]
    ymax = masked_img.shape[1]
    
    masked_img = masked_img > 0.9
    temp_masked_img = masked_img.copy()

    for i in range(ymax):
        for j in range(xmax):
            if temp_masked_img[i,j] == True:
                x_left, x_right = max(0,j-boundary), min(xmax-1, j+boundary)
                y_down, y_up = max(0,i-boundary), min(ymax-1, i+boundary)
                masked_img[y_down:y_up, x_left:x_right] = 1

    original_img = A.Resize(224,224)(image=np.array(original_img))['image']
    masked_img = np.expand_dims(masked_img, axis = 2)
    masked_img = np.concatenate((masked_img, masked_img, masked_img), axis=2)
    
    return original_img * masked_img

In [22]:
cnt = 0
def train_segment():
    for person_dir in os.listdir(orig_img_dir_path):
        for fname in os.listdir(Path(orig_img_dir_path) / person_dir):
            img_full_path = Path(orig_img_dir_path) / person_dir / fname
            img = Image.open(img_full_path)
            segmented_img = Image.fromarray(segment_img(img))

            new_dir = Path(segmented_img_dir_path) / person_dir
            if not os.path.isdir(new_dir):
                os.makedirs(new_dir)
            new_path = new_dir / fname
            segmented_img.save(new_path)
        global cnt
        cnt += 1
        if cnt > 5:
            break

def eval_segment():
    for fname in os.listdir(Path(orig_img_dir_path)):
        img_full_path = Path(orig_img_dir_path) / fname
        img = Image.open(img_full_path)
        segmented_img = Image.fromarray(segment_img(img))
        
        new_dir = Path(segmented_img_dir_path)
        if not os.path.isdir(new_dir):
            os.makedirs(new_dir)
        
        new_path = new_dir / fname
        segmented_img.save(new_path)
        break

In [23]:
if train_dir:
    train_segment()
else:
    eval_segment()

In [604]:
# test_person_seg_dir = Path(segmented_img_dir_path) / '003106_female_Asian_20'
# test_person_orig_dir = Path(orig_img_dir_path) / '003106_female_Asian_20'

# cnt = 1
# plt.figure(figsize=(50,20))

# for seg_fname in os.listdir(test_person_seg_dir):
#     img = Image.open(test_person_seg_dir / seg_fname)
#     plt.subplot(2,7,cnt)
#     plt.imshow(img)
#     plt.axis('off')
#     cnt += 1
    
# for seg_fname in os.listdir(test_person_orig_dir):
#     img = Image.open(test_person_orig_dir / seg_fname)
#     plt.subplot(2,7,cnt)
#     plt.imshow(img)
#     plt.axis('off')
#     cnt += 1
    
# plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'eval/images_segmented/003106_female_Asian_20'

<Figure size 3600x1440 with 0 Axes>